# Transforming the OpenEI original dataset into usable data files

This notebook shows how to transform the original (full) [OpenEI dataset](https://data.openei.org/submissions/153)
into the set of binary data files (.npz) in the `data/openei` folder, which
can be loaded by our code.

It serves several purposes:

1. Transparency: document what is done, so that researchers can agree (or not).
2. Reproducibility: allow researchers to re-create the files if necessary,
   including with some tweaks (e.g., changing the max_storage value).
3. Example: this notebook could serve as an example to take inspiration from,
   when adapting a new dataset.

## Step 1. Download files


In [ ]:
# The data is in 2 parts: commercial and residential.
!curl -o commercial.tar.zip https://data.openei.org/files/153/COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT.tar.zip
!curl -o residential.zip https://data.openei.org/files/153/RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT.zip

## Step 2. Uncompress data

In [ ]:
household_filename = 'USA_AK_Anchorage.Intl.AP.702730_TMY3_BASE.csv'
office_filename = 'RefBldgSmallOfficeNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv'
school_filename = 'RefBldgPrimarySchoolNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv'

In [ ]:
# We want a single file, we do not need the full archive content.
# `-j`: junk paths, does not re-create the archive hierarchy.
# Only the "RES.../$household_filename" file will be extracted, to the current
# directory.
!unzip -j residential.zip "RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT/BASE/"$household_filename

# It is a bit harder with the commercial archive, as it is a ZIP archive of
# several parts (part1, part2, ...) of TAR.GZ archives.
!unzip commercial.tar.zip
!tar xzf COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT.part1.tar.gz --strip-components=1 "USA_AK_Anchorage.Intl.AP.702730_TMY3/"$office_filename
!tar xzf COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT.part1.tar.gz --strip-components=1 "USA_AK_Anchorage.Intl.AP.702730_TMY3/"$school_filename

## Step 3. Parse data

We want to build 3 agent profiles:

- a Household (corresponding to the `household_filename` CSV file);
- an Office (`office_filename` CSV file);
- a School (`school_filename` CSV file).


In [ ]:
import pandas as pd

def parse_data(filepath):
    df = pd.read_csv(filepath)
    # We need to split Date and Time (just in case we want to aggregate)
    df[['Date', 'Time']] = df['Date/Time'].str.split('  ', n=1, expand=True, regex=False)
    # Transform Electricity from kW to W, and round to integer
    df['Electricity (Wh)'] = df['Electricity:Facility [kW](Hourly)'] * 1000
    df['Electricity (Wh)'] = df['Electricity (Wh)'].astype(int)
    # Keep only the `Date/Time` and `Electricity (Wh)` columns
    df = df[['Date', 'Time', 'Electricity (Wh)']]
    return df

def annual_to_daily(df):
    # To have a single datapoint per hour, we want to aggregate all days.
    # So we group by `Time` (the hour), and take the average.
    return df['Electricity (Wh)'].groupby('Time')['Electricity (Wh)'].agg('mean')

In [ ]:
df_household = parse_data(household_filename)
df_office = parse_data(office_filename)
df_school = parse_data(school_filename)

## Step 4. Write data profiles

Now that we have the profiles' needs (electricity consumed each hour), we
will write the profiles to data files, which are NumPy archive (npz files).

In [ ]:
import numpy as np

def write_profile(name, needs, max_storage, action_limit):
    filepath = f'../data/openei/{name}'
    np.savez(filepath,
             needs=needs,
             action_limit=action_limit,
             max_storage=max_storage)


In [ ]:
# Residential - Annual profile
write_profile(
    'profile_residential_annually.npz',
    df_household['Electricity (Wh)'],
    500,
    2500  # This is slightly higher than `df_household['Electricity (Wh)'].max()`
)

In [ ]:
# Residential - Daily profile
write_profile(
    'profile_residential_daily.npz',
    annual_to_daily(df_household),
    500,
    2000
)

In [ ]:
# Office - Annual profile
write_profile(
    'profile_office_annually.npz',
    df_office['Electricity (Wh)'],
    2500,
    14100
)

In [ ]:
# Office - Daily profile
write_profile(
    'profile_office_daily.npz',
    annual_to_daily(df_office),
    2500,
    11000
)

In [ ]:
# School - Annual profile
write_profile(
    'profile_school_annually.npz',
    df_school['Electricity (Wh)'],
    10_000,
    205_000
)

In [ ]:
# School - Daily profile
write_profile(
    'profile_school_daily.npz',
    annual_to_daily(df_school),
    10_000,
    125_000
)